In [1]:
import numpy as np
import ast

from modules.helper_functions_tsp import ( 
    read_file_name, validate_distance_array, find_problem_size, cost_fn_fact, 
    read_index, hot_start, hot_start_list_to_string,
    update_parameters_using_gradient, define_parameters, create_initial_rotations,
    bind_weights, vqc_circuit, cost_func_evaluate, find_run_stats)
from classes.DataLogger import DataLogger, SubDataLogger

import copy
import time

from modules.config import(CONTROL_FILE, ENCODING,
                           DATA_SOURCES, CACHE_MAX_SIZE
                           )

Load control data and instantiate data logger

In [2]:
control_dict = read_index(CONTROL_FILE, ENCODING)
print(control_dict)
datalogger = DataLogger()

{0: {'locations': '4', 'slice': '1', 'shots': '1024', 'mode': '2', 'iterations': '50', 'gray': 'True', 'hot_start': 'True', 'gradient_type': 'SPSA', 'formulation': 'new', 'alpha': '0.602', 'big_a': '50', 'c': '0.314', 'eta': '0.02', 'gamma': '0.101', 's': '0.5', 'print_frequency': '50'}, 1: {'locations': '4', 'slice': '1', 'shots': '1024', 'mode': '2', 'iterations': '50', 'gray': 'False', 'hot_start': 'False', 'gradient_type': 'SPSA', 'formulation': 'new', 'alpha': '0.602', 'big_a': '50', 'c': '0.314', 'eta': '0.02', 'gamma': '0.101', 's': '0.5', 'print_frequency': '50'}}
Data logger instantiated.  Run ID: 20250216-16-00-16


## Main loop

In [3]:

for keys, control_items in control_dict.items():
    subdatalogger = SubDataLogger(datalogger)
    data_dict = dict(control_items)

    locations = int(data_dict['locations'])
    slice = float(data_dict['slice'])
    shots = int(data_dict['shots'])
    mode = int(data_dict['mode'])
    iterations = int(data_dict['iterations'])
    #gray = bool(data_dict['gray'])
    gray = ast.literal_eval(data_dict['gray'])
    #hot_start_bool = bool(data_dict['hot_start'])
    hot_start_bool = ast.literal_eval(data_dict['hot_start'])
    gradient_type = data_dict['gradient_type']
    formulation = data_dict['formulation']
    alpha = float(data_dict['alpha'])
    big_a = float(data_dict['big_a'])
    c = float(data_dict['c'])
    eta = float(data_dict['eta'])
    gamma = float(data_dict['gamma'])
    s = float(data_dict['s'])
    print_frequency = int(data_dict['print_frequency'])

    data_dict['runid'] = datalogger.runid
    data_dict['subid'] = subdatalogger.subid
    data_dict['cache_max_size'] = CACHE_MAX_SIZE

    qubits = find_problem_size(locations, formulation)
    data_filename = read_file_name(locations, DATA_SOURCES)
    #Data sources are held locally to avoid downstream dependencies.  
    #Read the data, and print out the filename and best distance held in the data.
    best_dist = DATA_SOURCES[locations]['best']
    data_dict['best_dist'] = best_dist
    #Read and validate the distance array.  This checks the array is the correct shape, and is symmetric.
    distance_array = np.genfromtxt(data_filename)
    validate_distance_array(distance_array, locations)
    #Define the VQC circuits with appropriate parameters
    params = define_parameters(qubits, mode)
    #define the cost function for this run
    qc = vqc_circuit(qubits, params, mode)
    cost_fn = cost_fn_fact(locations,distance_array, gray, method = formulation)

    print(hot_start_bool)

    if hot_start_bool:
        hot_start_list = hot_start(distance_array, locations)
        bin_hot_start_list =  hot_start_list_to_string(hot_start_list, locations, gray, formulation)
        hot_start_dist = cost_fn(bin_hot_start_list)
        init_rots = create_initial_rotations(qubits, mode, bin_hot_start_list, hot_start=True)
    else:
        init_rots = create_initial_rotations(qubits, mode)

    bc = bind_weights(params, init_rots, qc)
    bc.measure_all()

    if hot_start_bool:
        hot_start_dist, _, _ = cost_func_evaluate(cost_fn, bc, shots=shots, average_slice=slice)
        data_dict['hot_start_dist'] = hot_start_dist
    else:
        data_dict['hot_start_dist'] = 'n/a'

    t0 = time.time()
    av_cost_list_all, lowest_list_all, sliced_cost_list_all = [], [], []
    rots = copy.deepcopy(init_rots)
    
    index_list, sliced_list, lowest_list, _ , average_list, _ = \
    update_parameters_using_gradient(locations=locations, iterations=iterations, 
                                    print_frequency=print_frequency, params=params,
                                    rots=rots,  
                                    cost_fn=cost_fn, qc = qc, shots=shots, s=s, 
                                    eta=eta, average_slice=slice, gray=gray, 
                                    verbose=False, gradient_type=gradient_type,
                                    alpha=alpha, gamma=gamma, c=c,
                                    big_a=big_a,
                                    method=formulation,
                                    print_results=False
            )
    av_cost_list_all.append(average_list)
    lowest_list_all.append(lowest_list) 
    sliced_cost_list_all.append(sliced_list)
    best_dist_found, iteration_found = find_run_stats(lowest_list)
    data_dict['best_dist_found'] = best_dist_found
    data_dict['iteration_found'] = iteration_found
    t1 = time.time()
    elapsed = t1-t0
    data_dict['elapsed'] = elapsed

    items, hits, misses = cost_fn.report_cache_stats()
    data_dict['cache_items'] = items
    data_dict['cache_hits'] = hits
    data_dict['cache_misses'] = misses
    #data_dict['cache_items'] = len(cost_fn.cache)
    #data_dict['cache_hits'] = cost_fn.cache_hits
    #data_dict['cache_misses'] = cost_fn.cache_misses

    cost_fn.clear_cache() #need to clear cache so statistics are not cumulative
    
    subdatalogger.save_dict_to_csv(data_dict) # don't write header


Data logger instantiated.  Run ID: 20250216-16-00-16
SubDataLogger instantiated.  Run ID = 20250216-16-00-16 - 16-00-16
Folder data_sub_path = data\20250216-16-00-16 is used for data writing
True
Data for Run ID: 20250216-16-00-16 - 16-00-16 successfully added to data\20250216-16-00-16\20250216-16-00-16.csv
Data logger instantiated.  Run ID: 20250216-16-00-18
SubDataLogger instantiated.  Run ID = 20250216-16-00-16 - 16-00-18
Folder data_sub_path = data\20250216-16-00-16 is used for data writing
False
Data for Run ID: 20250216-16-00-16 - 16-00-18 successfully added to data\20250216-16-00-16\20250216-16-00-16.csv
